In [16]:
# 匯入套件
from bs4 import BeautifulSoup as bs
import requests as req
from pprint import pprint
import time

In [17]:
# 取得台北市立圖書館常見問答網址
# url = "https://tpml.gov.taipei/News.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&page=1&PageSize=20" 
# url = "https://tpml.gov.taipei/News.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&page=2&PageSize=20"
url = "https://tpml.gov.taipei/News.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&page=3&PageSize=20"  

# 用 requests 的 get 方法把網頁抓下來
res = req.get(url) 

# 指定 lxml 作為解析器
soup = bs(res.text, "lxml") 

# 建立 list 來放置列表資訊
list_posts = []

In [18]:
# 清空放置列表資訊的變數
list_posts.clear()

# 取得列表的文字與超連結
for td in soup.select('td.CCMS_jGridView_td_Class_1'):
    # 使用 select 找到 <a> 標籤
    a_tags  = td.select('a')
    for a in a_tags:
        print(a.get_text())  # 列印文字
        print(a['href'])     # 列印超連結  或是 a.get('href')


    # 加入列表資訊
    list_posts.append({
        'question': a.get_text(),
        'link': 'https://tpml.gov.taipei/' + a['href']
    })

申請件如何領取?
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=AECFF2C5F5140AFB
本館沒有收藏的期刊文獻，是否可提出館際複印，申請他館期刊文獻複印服務?
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=1683335362F92507
館際合作的服務項目有哪些?
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=F23E6AF8467B69DC
圖書狀態顯示在「新到圖書區」的圖書可以預約嗎？
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=C38829A5E30E7CBC
線上預約申請可以取消或修改取書館嗎？
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=A3824D484BD96E05
預約書如何通知？
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=B44E47F0808604FB
圖書一定要在原借書館歸還嗎？
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=DC115A94B9DE2CAB
圖書資料遺失或毀損，該如何賠償?
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=5C13DE8CB1694B9C
借閱證可以請人代借嗎?
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=5B49D47BCC88ADC7
悠遊卡借閱證中晶片之資料是否會造成讀者資料外洩？
News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=94E37F38693B1543
換了悠遊卡借閱證是否可進入智慧圖書館及在各分館借書？
News_Content.aspx?n=E5F

In [19]:
# 走訪每一個 a link，整合網頁內文
for index, obj in enumerate(list_posts):
    res_ = req.get(obj['link'])
    soup_ = bs(res_.text, "lxml")
    
    # 取得實際需要的內容 (類似 JavaScript 的 innerHTML)
    html = soup_.select_one('p').decode_contents()
    # print(html)
    
    # 使用 BeautifulSoup 解析並清理內容
    clean_soup = bs(html, "lxml")
    
    # 找到所有的 <p> 標籤，並提取其文字
    paragraphs = clean_soup.select('p')
    # print(paragraphs)
    content = "".join([p.get_text() for p in paragraphs])
    # print(clean_text)

    # 預覽列表和內文
    # print(obj['title'])
    # print(obj['link'])
    # print(clean_text)
    print(f"問: {obj['question']} \n答: {content} \n網址: {obj['link']} ")
    print("=" * 50)
    
    # 整合到列表資訊的變數當中
    list_posts[index]['answer'] = content

# 預覽所有結果
pprint(list_posts)

問: 申請件如何領取? 
答: 館際合作申借之圖書或文件到館時，本館即以申請者所登錄的E-mail或電話通知，請申請者辦理繳費及取件！若因故無法提供圖書或文件資料之申借服務時，本館亦會以E-mail或電話通知。 
網址: https://tpml.gov.taipei/News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=AECFF2C5F5140AFB 
問: 本館沒有收藏的期刊文獻，是否可提出館際複印，申請他館期刊文獻複印服務? 
答: 可以。您可利用「全國文獻傳遞服務系統」，查詢所需之期刊聯合目錄後，自行於線上申請「向國內申請複印」或「向國外申請複印」。或向館員索取「臺北市立圖書館館際合作申請單」填寫完畢後，直接交給館員代為申請。但申請前有幾點請注意：1. 確認本館館藏確實沒有：請先進入本館館藏目錄系統中查詢。2. 申請本館館際合作服務前，必須先辦理本館之借閱證後，始得申請。3. 若您是辦理線上申請，您必須先申請讀者個人帳號，經過系統認證後，始可透過「全國文獻傳遞服務系統」申請資料。 
網址: https://tpml.gov.taipei/News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=1683335362F92507 
問: 館際合作的服務項目有哪些? 
答: 代為向國內圖書館辦理圖書借閱及期刊複印、國外圖書館及上海圖書館複印服務。 
網址: https://tpml.gov.taipei/News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&s=F23E6AF8467B69DC 
問: 圖書狀態顯示在「新到圖書區」的圖書可以預約嗎？ 
答: 新到圖書區的圖書同樣接受讀者線上預約，然為保障到館讀者借閱新到圖書之權利，本館新書加工處理後，均先於館內新書架上陳列，7日後未被外借方提供架上預約圖書查找服務並調撥至讀者選定之取書館，此係避免熱門新書未及於館內上架即被預約讀者借閱一空，造成到館讀者無新書可借之情況。 
網址: https://tpml.gov.taipei/News_Content.aspx?n=E5F579B94C9D2941&sms=87415A8B

In [ ]:
# 自行輸入頁數.取得所有問答內容與網址
def fetch_liberary_data(page):
    # 建立 list 來放置列表資訊
    list_posts = []
    
    # 取得新聞列表
    url = f"https://tpml.gov.taipei/News.aspx?n=E5F579B94C9D2941&sms=87415A8B9CE81B16&page={page}&PageSize=20" 

    # 用 requests 的 get 方法把網頁抓下來
    res = req.get(url) 

    # 指定 lxml 作為解析器
    soup = bs(res.text, "lxml") 

    # 取得列表的文字與超連結
    for td in soup.select('td.CCMS_jGridView_td_Class_1'):
    # 使用 select 找到 <a> 標籤
        a_tags  = td.select('a')
        for a in a_tags:
        # 加入列表資訊
            list_posts.append({
                'question': a.get_text(),
                'link': 'https://tpml.gov.taipei/' + a['href']
                })
            
        # 走訪每一個 a link，整合網頁內文
        for index, obj in enumerate(list_posts):
            res_ = req.get(obj['link'])
            soup_ = bs(res_.text, "lxml")

            # 取得實際需要的內容 (類似 JavaScript 的 innerHTML)
            html = soup_.select_one('p').decode_contents()

            # 使用 BeautifulSoup 解析並清理內容
            clean_soup = bs(html, "lxml")

            # 找到所有的 <p> 標籤，並提取其文字
            paragraphs = clean_soup.select('p')
            content = "".join([p.get_text() for p in paragraphs])
            
            # 整合到列表資訊的變數當中
            list_posts[index]['html'] = content

            # 預覽所有結果
            print(f"問: {obj['question']} \n 答: {content} \n 網址:{obj['link']} ")
            # pprint(list_posts)

        return list_posts  # 返回所有的資料
    
# 自行輸入頁數 顯示該頁資料
page_number = int(input("請輸入頁數: "))
fetch_liberary_data(page_number)

# # 起始頁數
# init_page = 1

# # 最新頁數
# latest_page = 2

# # 在已經知道分頁數的情況下
# for page in range(init_page, latest_page + 1):
#     print(f"正在處理第 {page} 頁...")
#     fetch_liberary_data(page)  # 獲取當前頁的所有問答資料